<img src="https://farm66.staticflickr.com/65535/49055715328_092031af74_o.png"  width="150" />

<font size="7"> Selection Script
    
<font size="4">
 This notebook was developed by Phelipe Darc. Its a notebook tutorial of how to create a Text dataset by selecting phrases by Keywords and saving it into a CSV file.

    Date: 22/12/22
    
**Contact:** phelipedarc@gmail.com</font>

<font size="4">

Please do not remove this disclaimer.</font>

### Importing

In [3]:
import os
import time
import pandas as pd
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
import h5py
import numpy as np

### Script Function : 
 - This function take as input a list of texts, splits into phrases and  select and save the phrases in which any of the keywords appears, and finally marks the corresponding collumn of the keyword that appears with the number one.

In [4]:
import re
def extract_wordtestpronta(text,words,multiwords=False):
    ''' A função pega um texto e separa em frases terminadas em ponto final.
    Depois se multiwortds=False então ele pega as frases contendo uma palavra ou mais palavras 
    Para usar a ou usar o simbolo "word1|word2".
    Caso multiwords=True então ele pega as frases que contém todas as palavras listadas 
    Cada Keyword corresponde a uma coluna, se a keyword aparecer a coluna é marcada com o numero 1, caso contrário é marcada com zero (0) 
    Além disso há uma coluna chamada '**negative**' que acusa se a frase contém adverbios de negação'''
    if multiwords==False:
        keywords = words.split('|')
        texto = pd.DataFrame(text['phrases'])
        text_filter = texto.loc[texto['phrases'].str.contains('(?i)'+words, regex=True,na=False,case = False)]
        
    else:
        keywords = words.split('&')
        words = words.split('&')
        texto = pd.DataFrame(text['phrases'])
        tamanho=len(words)
        b = texto['phrases'].apply(lambda sentence: all(word in sentence for word in words))
        text_filter = texto[b]        
    notappear = pd.DataFrame.copy(text_filter.reset_index(drop=True))
    notappear.rename(columns={'phrases': 'Negative'}, inplace=True)
    for l in range(len(notappear)):
        if "não" in notappear['Negative'][l]:
            notappear['Negative'][l]="True"
        else:
            notappear['Negative'][l]="False"
    finalfinal=text_filter.reset_index(drop=False)   
    for l in range(len(keywords)):
        finalfinal[str(keywords[l])] = 1*np.logical_or.reduce([finalfinal[col].str.contains(keywords[l],regex=True,case = False) 
                                          for col in ['phrases']])
        
    #finaldataset = pd.concat((text_filter,text_prepro,notappear),axis=1)
    finaldataset = pd.concat((finalfinal,notappear),axis=1)
    finaldataset['True_True'] = 0
     
    return finaldataset

## Importing a list of comments from different youtube videos

## Exemple of the disease - 'ASMA':

In [7]:
data1= pd.read_csv('/tf/phelipedock/Lemobs/Dataset_asma/asmavideo1.csv')
data2= pd.read_csv('/tf/phelipedock/Lemobs/Dataset_asma/asmavideo2.csv')
data3= pd.read_csv('/tf/phelipedock/Lemobs/Dataset_asma/asmavideo3.csv')
data4= pd.read_csv('/tf/phelipedock/Lemobs/Dataset_asma/asmavideo4.csv')
data5= pd.read_csv('/tf/phelipedock/Lemobs/Dataset_asma/asmavideo5.csv')
data6= pd.read_csv('/tf/phelipedock/Lemobs/Dataset_asma/asmavideo6.csv')

In [9]:
datafinal = pd.concat((data1,data2,data3,data4,data5,data6))

##### Renaming the columns

In [10]:
datafinal.rename(columns={'comments': 'phrases'}, inplace=True)

In [17]:
datafinal

,phrases
0,Sim e isso mesmo
1,"Eu sou asmática, faço tratamento com Alenia e ..."
2,Nem eu nem meu marido tem asma mas nossa filha...
3,"Quando puder, faça vídeo separado sobre Endoca..."
4,"Adorei, ficou claro agora tudo que li."
...,...
189,EU NÃO SEI SE TENHO ASMAS MAS EU TENHO MUITA A...
190,crb eu iria ficar mto feliz se o dia em invent...
191,EU TENHO ASMA E ME SINTO COMO UMA PESSOA NÃO N...
192,eu ouvi falar de uma tal vacina para a cura da...


In [10]:
#Choosing the symptoms of exemple disease and separating each with ' | '
palavrasasma='asmatico|asmático|asma|ar|falta de ar|dificuldade para respirar|aperto no peito|peito pesado|chio no peito|tosse'

##### Using the script to select the phrases of that contains the Keywords

In [28]:
dataasma=extract_wordtestpronta(datafinal,palavrasasma,multiwords=False)

In [29]:
dataasma

,index,phrases,asmatico,asmático,asma,ar,falta de ar,dificuldade para respirar,aperto no peito,peito pesado,chio no peito,tosse,Negative,True_True
0,1,"Eu sou asmática, faço tratamento com Alenia e ...",0,0,0,1,0,0,0,0,0,0,False,0
1,2,Nem eu nem meu marido tem asma mas nossa filha...,0,0,1,1,0,0,0,0,0,0,True,0
2,3,"Quando puder, faça vídeo separado sobre Endoca...",0,0,0,1,0,0,0,0,0,0,False,0
3,4,"Adorei, ficou claro agora tudo que li.",0,0,0,1,0,0,0,0,0,0,False,0
4,6,Eu tenho alergia fico com faltando ar,0,0,0,1,0,0,0,0,0,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,189,EU NÃO SEI SE TENHO ASMAS MAS EU TENHO MUITA A...,0,0,1,1,1,0,0,0,0,0,False,0
965,190,crb eu iria ficar mto feliz se o dia em invent...,0,0,1,1,0,0,0,0,0,0,False,0
966,191,EU TENHO ASMA E ME SINTO COMO UMA PESSOA NÃO N...,0,0,1,1,0,0,0,0,0,0,False,0
967,192,eu ouvi falar de uma tal vacina para a cura da...,0,0,1,1,0,0,0,0,0,0,False,0


In [34]:
dataasma.to_excel('/tf/phelipedock/Dataset_asma/RAW_asmadataset.xlsx')  

## Arritimia

In [35]:
data1= pd.read_csv('/tf/phelipedock/Dataset_arritimia_fibrilacao/arritimiavideo1.csv')
data2= pd.read_csv('/tf/phelipedock/Dataset_arritimia_fibrilacao/arritimiavideo2.csv')
data3= pd.read_csv('/tf/phelipedock/Dataset_arritimia_fibrilacao/arritimiavideo3.csv')

In [36]:
datafinal = pd.concat((data1,data2,data3))

In [37]:
datafinal.rename(columns={'comments': 'phrases'}, inplace=True)

In [38]:
datafinal

,phrases
0,tenho falta de ar e coracao acelerado quando d...
1,Estou surda com dificuldade de respirar tive ...
2,Meu coração começa bater muito forte derepente...
3,Doutor você passa uma calma profunda meu obrigado
4,Bom dia ! Doutor .\nEu comecei a sentir uns si...
...,...
11,Boa.noite eu tenho fibrilacao atrial tive avc .
12,Muito bom esse esclarecimento
13,É só uma fibrilaçãozinha.
14,"Faz um vídeo de fibrilação ventricular, com im..."


In [39]:
#palavrasasma = 'Diabete|Diabetes|pele|rim|Fome frequente|Sede|Vontade de urinar|Perda de peso|Fraqueza|Fadiga|humor|Náusea|vômito|infecções'
palavras='fibrilação|arritimia|cansaço|palpitações|desmaios|tontura|confusão mental|fraqueza|pressão baixa|dor no peito'

In [40]:
data0=extract_wordtestpronta(datafinal,palavras,multiwords=False)

In [41]:
data0

,index,phrases,fibrilação,arritimia,cansaço,palpitações,desmaios,tontura,confusão mental,fraqueza,pressão baixa,dor no peito,Negative,True_True
0,1,Estou surda com dificuldade de respirar tive ...,0,1,0,0,0,0,0,0,0,0,False,0
1,27,Dr.tenho problema serio de coluna tipo osteopo...,0,0,0,0,0,0,0,0,0,1,False,0
2,29,Oi boa noite pode me tirar uma duvida eu f...,0,0,1,0,0,0,0,0,0,0,False,0
3,33,Tenho palpitações direto e perigoso?,0,0,0,1,0,0,0,0,0,0,False,0
4,42,Gosto Muito desse medico adoro ele falando sob...,0,1,0,0,0,0,0,0,0,0,False,0
5,55,eu tenho fibrilação e horrível,1,0,0,0,0,0,0,0,0,0,False,0
6,61,Dr. A fibrilação atrial têm haver com hiperten...,1,0,0,0,0,0,0,0,0,0,False,0
7,114,"Meu pai apresentava cansaços, inchaços nos pés...",0,0,1,0,0,0,0,0,0,0,True,0
8,116,Eu tinha arritmia é dia 29 agora fiz o estudo ...,0,1,0,0,0,0,0,0,0,0,False,0
9,0,Estou suspeitando que eu tenho fibrilação atri...,1,0,0,0,0,0,0,0,0,0,False,0


In [42]:
data0.to_excel('/tf/phelipedock/Dataset_arritimia_fibrilacao/RAW_arritimiadataset.xlsx')  

## Cancer

In [44]:
data1= pd.read_csv('/tf/phelipedock/Dataset_cancer/cancervideo1.csv')
data2= pd.read_csv('/tf/phelipedock/Dataset_cancer/cancervideo2.csv')
data3= pd.read_csv('/tf/phelipedock/Dataset_cancer/cancervideo3.csv')
data4= pd.read_csv('/tf/phelipedock/Dataset_cancer/cancervideo4.csv')
data5= pd.read_csv('/tf/phelipedock/Dataset_cancer/cancervideo5.csv')
data6= pd.read_csv('/tf/phelipedock/Dataset_cancer/cancervideo6.csv')
data7= pd.read_csv('/tf/phelipedock/Dataset_cancer/cancervideo7.csv')

In [45]:
datafinal = pd.concat((data1,data2,data3,data4,data5,data6,data7))

In [46]:
datafinal.rename(columns={'comments': 'phrases'}, inplace=True)

In [47]:
datafinal

,phrases
0,"Obrigada, Dr, pela explicação."
1,Hj em dia temos muito desruptores endócrinos. ...
2,Então resolvido !! Se pararmos de fumar e nos ...
3,Dr Dráuzio Varella*agora está correto
4,As vezes acho que deveria ter um Dr Dráusio Va...
...,...
87,Dráuzio fala sobre a canabis no tratamento do ...
88,"Sinto dores nos seios com frequência, isso é n..."
89,Vc é o cara Drauzião!
90,✨❤❤🌹🌹😘😘 Bom dia!


In [51]:
#palavrasasma = 'Diabete|Diabetes|pele|rim|Fome frequente|Sede|Vontade de urinar|Perda de peso|Fraqueza|Fadiga|humor|Náusea|vômito|infecções'
#palavras='fibrilação|arritimia|cansaço|palpitações|desmaios|tontura|confusão mental|fraqueza|pressão baixa|dor no peito'
palavras='cancer|Dor|Fadiga|Falta de apetite|Náuseas|vômitos|Edema|linfedema|Constipação intestinal|Obstrução intestinal|Alteração da mucosa oral|Diarreia|Aumento do volume abdominal|Sangramento|Depressão'

In [52]:
data0=extract_wordtestpronta(datafinal,palavras,multiwords=False)

In [53]:
data0

,index,phrases,cancer,Dor,Fadiga,Falta de apetite,Náuseas,vômitos,Edema,linfedema,Constipação intestinal,Obstrução intestinal,Alteração da mucosa oral,Diarreia,Aumento do volume abdominal,Sangramento,Depressão,Negative,True_True
0,2,Então resolvido !! Se pararmos de fumar e nos ...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,True,0
1,8,Drauzio Varella minha vó teve um endocarciroma...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,0
2,21,Dr. Jaime Bruning tinha receitas naturais no s...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,False,0
3,41,É verdade que o aborto aumenta o cancer de mama?,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,0
4,46,"o exercício físico regular, sobretudo a muscul...",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,62,"Doutor, espero que o senhor ou alguem que teve...",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,0
57,72,Drauzio fale sobre o papel do AÇÚCAR na progre...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,0
58,73,Cade o mês marrom dedicado ao cancer de intest...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,0
59,77,Minha cachorrinha está com cancer de mama ( ma...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,0


In [54]:
data0.to_excel('/tf/phelipedock/Dataset_cancer/RAW_cancerdataset.xlsx')  

## Cardiopatia:

In [55]:
data1= pd.read_csv('/tf/phelipedock/Dataset_cardiopatia/cardiopatiavideo1.csv')
data2= pd.read_csv('/tf/phelipedock/Dataset_cardiopatia/cardiopatiavideo2.csv')


In [56]:
datafinal = pd.concat((data1,data2))

In [57]:
datafinal.rename(columns={'comments': 'phrases'}, inplace=True)

In [58]:
datafinal

,phrases
0,"Dr. Odeio fazer força e amo pedalar, é necessá..."
1,Quando estou fazendo zumba ficou com o coração...
2,Drauzio eu fico andado na minha casa pulando d...
3,"Dr Drauzio,\nGrato, o Sr sempre com notável co..."
4,Dr eu corria antes de fazer uma cirurgia de co...
...,...
225,Tem horário pra tomar remédio da hipertensão?
226,Eliminei 9 medicamentos e 50 kls.da minha vid...
227,Dr. Drauzio Varella o senhor sem dúvidas é min...
228,Dr Dráuzio vc é a top


In [72]:
#palavrasasma = 'Diabete|Diabetes|pele|rim|Fome frequente|Sede|Vontade de urinar|Perda de peso|Fraqueza|Fadiga|humor|Náusea|vômito|infecções'
#palavras='fibrilação|arritimia|cansaço|palpitações|desmaios|tontura|confusão mental|fraqueza|pressão baixa|dor no peito'
palavras = 'cardio|cardiopatia|pele cinzenta|pele azul|cianose|Inchaço nas mãos|Inchaço nos tornozelos|Inchaço nos pés|Falta de ar|Falta de fôlego|Fadiga|Batimentos cardíacos irregulares|Tonturas|vertigens|desmaios|Dor no peito'

In [75]:
data0=extract_wordtestpronta(datafinal,palavras,multiwords=False)

In [76]:
data0.to_excel('/tf/phelipedock/Dataset_cardiopatia/RAW_cardiopatiadataset.xlsx')  

## Diabetes

In [78]:
data1= pd.read_csv('/tf/phelipedock/Dataset_diabetes/diabetesvideo1.csv')
data2= pd.read_csv('/tf/phelipedock/Dataset_diabetes/diabetesvideo2.csv')
data3= pd.read_csv('/tf/phelipedock/Dataset_diabetes/diabetesvideo3.csv')
data4= pd.read_csv('/tf/phelipedock/Dataset_diabetes/diabetesvideo4.csv')
data5= pd.read_csv('/tf/phelipedock/Dataset_diabetes/diabetesvideo5.csv')
data6= pd.read_csv('/tf/phelipedock/Dataset_diabetes/diabetesvideo6.csv')
data7= pd.read_csv('/tf/phelipedock/Dataset_diabetes/diabetesvideo7.csv')


In [79]:
datafinal = pd.concat((data1,data2,data3,data4,data5,data6,data7))

In [80]:
datafinal.rename(columns={'comments': 'phrases'}, inplace=True)

In [81]:
datafinal

,phrases
0,Tomo metformida e certo
1,Eu estou tomando o Insultrill e ja estou com a...
2,"ta com 2 semanas que tomo Insultrill, minha di..."
3,"Gostei do vídeo, uma dica que está ajudando mi..."
4,Descobrir que sou diabética tipo 2 não está se...
...,...
32,Obrigado esse diabetes tipo 1 que o senhor fal...
33,Esse cara é tão bom no que faz que precisa ler...
34,Man müsste eine Tablette erfinden -die so wirk...
35,Bom dia tem algum vídeo com relação a diabetes...


In [82]:
palavras =' Diabete|Diabetes|pele|rim|Fome frequente|Sede|Vontade de urinar|Perda de peso|Fraqueza|Fadiga|humor|Náusea|vômito|infecções'

In [83]:
data0=extract_wordtestpronta(datafinal,palavras,multiwords=False)

In [84]:
data0

,index,phrases,Diabete,Diabetes,pele,rim,Fome frequente,Sede,Vontade de urinar,Perda de peso,Fraqueza,Fadiga,humor,Náusea,vômito,infecções,Negative,True_True
0,1,Eu estou tomando o Insultrill e ja estou com a...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,False,0
1,2,"ta com 2 semanas que tomo Insultrill, minha di...",1,0,0,0,0,0,0,0,0,0,0,0,0,0,False,0
2,7,"Boa tarde! Quem tem diabetes tipo 2, pode toma...",1,1,0,0,0,0,0,0,0,0,0,0,0,0,False,0
3,9,D varela diabete tipo 2 tem cura???,1,0,0,0,0,0,0,0,0,0,0,0,0,0,False,0
4,15,Bom dia para todos Vocês \n Eu tenho diabetes ...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,25,Dr meu filho tem diabetes está com tosse não s...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,True,0
367,26,"Olá, gostaria de ajudar as pessoas que possuem...",1,1,0,0,0,0,0,0,0,0,0,0,0,0,True,0
368,32,Obrigado esse diabetes tipo 1 que o senhor fal...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,False,0
369,35,Bom dia tem algum vídeo com relação a diabetes...,1,1,0,0,0,0,0,0,0,0,0,0,0,0,False,0


In [85]:
data0.to_excel('/tf/phelipedock/Dataset_diabetes/RAW_diabetesdataset.xlsx')  

## Pressão Alta

In [86]:
data1= pd.read_csv('/tf/phelipedock/Dataset_pressaoalta/pressaoaltavideo1.csv')
data2= pd.read_csv('/tf/phelipedock/Dataset_pressaoalta/pressaoaltavideo2.csv')
data3= pd.read_csv('/tf/phelipedock/Dataset_pressaoalta/pressaoaltavideo3.csv')
data4= pd.read_csv('/tf/phelipedock/Dataset_pressaoalta/pressaoaltavideo4.csv')


In [87]:
datafinal = pd.concat((data1,data2,data3,data4))

In [88]:
datafinal.rename(columns={'comments': 'phrases'}, inplace=True)

In [89]:
datafinal

,phrases
0,"Agradecida, Sr. Dr. Drauzio."
1,DOUTOR DRÁUZIO EXPLICA DE MANEIRA SIMPLES E M...
2,Boa noite tenho 43 anos a um ano descubri q te...
3,Dr.a pessoa que tomou morfina\nCom quanto temp...
4,Meu Deus sou tão nova pra morrer😔
...,...
21,Excelente explicação!
22,Boa informação. Controle a pressão alta com Pr...
23,doutor queria saber por que minha pressão só a...
24,esclarecedor eu n entedia isso d 12_8 agora ap...


In [115]:
palavras = 'hiper|pressão|alta'

In [116]:
data0=extract_wordtestpronta(datafinal,palavras,multiwords=False)

In [117]:
data0

,index,phrases,hiper,pressão,alta,Negative,True_True
0,2,Boa noite tenho 43 anos a um ano descubri q te...,1,0,1,False,0
1,6,Tenho 32 anos e minha pressão esta doida o dia...,0,1,0,False,0
2,7,Drauzio Varella eu tenho 21 anos e minha press...,0,1,0,False,0
3,0,Estou deitada muita do nunca ja Tomie remédio ...,0,1,0,False,0
4,3,Pressao alta pode causar depressao choro trist...,0,0,1,False,0
...,...,...,...,...,...,...,...
127,18,"Dr Drauzio,será que o senhor pode explicar, ca...",0,1,1,True,0
128,19,Acabe com A HIPERTENSÃO \nMétodo caseiro\nhtt...,1,1,1,False,0
129,20,Aprenda a controlar a hipertensão com um livro...,1,1,1,False,0
130,22,Boa informação. Controle a pressão alta com Pr...,0,1,1,False,0


In [118]:
data0.to_excel('/tf/phelipedock/Dataset_pressaoalta/RAW_hipertensaodataset.xlsx')  